In [2]:
from pytorch_forecasting.data.examples import get_stallion_data
import pyarrow
import fastparquet
import numpy as np
import torch

In [3]:
data = get_stallion_data()

In [3]:
data.head()

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries
0,Agency_22,SKU_01,52.2720,2013-01-01,492612703,718394219,25.845238,1168.903668,1069.166193,99.737475,...,0,0,0,0,0,0,0,0,8.532566,0
238,Agency_37,SKU_04,0.0000,2013-01-01,492612703,718394219,26.505000,1852.273642,1611.466298,240.807344,...,0,0,0,0,0,0,0,0,13.000635,5
237,Agency_59,SKU_03,812.9214,2013-01-01,492612703,718394219,22.219737,1270.795012,1197.184260,73.610752,...,0,0,0,0,0,0,0,0,5.792496,9
236,Agency_11,SKU_01,316.4400,2013-01-01,492612703,718394219,25.360000,1176.155397,1082.757488,93.397909,...,0,0,0,0,0,0,0,0,7.940950,14
235,Agency_05,SKU_05,420.9093,2013-01-01,492612703,718394219,24.079012,1327.003396,1207.822992,119.180404,...,0,0,0,0,0,0,0,0,8.981168,22


In [4]:
# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month

data["time_idx"] -= data["time_idx"].min()
# add additional features

# show sample data
data.sample(10, random_state=521)

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries,time_idx
291,Agency_25,SKU_03,0.5076,2013-01-01,492612703,718394219,25.845238,1264.162234,1152.473405,111.688829,...,0,0,0,0,0,0,0,8.835008,228,0
871,Agency_29,SKU_02,8.7480,2015-01-01,498567142,762225057,27.584615,1316.098485,1296.804924,19.293561,...,0,0,0,0,0,0,0,1.465966,177,24
19532,Agency_47,SKU_01,4.9680,2013-09-01,454252482,789624076,30.665957,1269.250000,1266.490490,2.759510,...,1,0,0,0,0,0,0,0.217413,322,8
2089,Agency_53,SKU_07,21.6825,2013-10-01,480693900,791658684,29.197727,1193.842373,1128.124395,65.717978,...,0,0,0,0,0,1,0,5.504745,240,9
9755,Agency_17,SKU_02,960.5520,2015-03-01,515468092,871204688,23.608120,1338.334248,1232.128069,106.206179,...,0,0,0,0,0,0,1,7.935699,259,26
7561,Agency_05,SKU_03,1184.6535,2014-02-01,425528909,734443953,28.668254,1369.556376,1161.135214,208.421162,...,0,0,0,0,0,0,0,15.218151,21,13
19204,Agency_11,SKU_05,5.5593,2017-08-01,623319783,1049868815,31.915385,1922.486644,1651.307674,271.178970,...,0,0,0,0,0,0,0,14.105636,17,55
8781,Agency_48,SKU_04,4275.1605,2013-03-01,509281531,892192092,26.767857,1761.258209,1546.059670,215.198539,...,0,0,0,0,0,0,1,12.218455,151,2
2540,Agency_07,SKU_21,0.0000,2015-10-01,544203593,761469815,28.987755,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,300,33
12084,Agency_21,SKU_03,46.3608,2017-04-01,589969396,940912941,32.478910,1675.922116,1413.571789,262.350327,...,0,0,0,0,0,0,0,15.654088,181,51


In [5]:
data['time_idx'].max()

59

In [6]:
data.loc[(data['agency'] == 'Agency_22') & (data['sku'] == 'SKU_01')].\
    sort_values(by = 'time_idx')[['agency', 'sku', 'date', 'volume']].head()

,agency,sku,date,volume
0,Agency_22,SKU_01,2013-01-01,52.272
7096,Agency_22,SKU_01,2013-02-01,62.532
8898,Agency_22,SKU_01,2013-03-01,74.196
10733,Agency_22,SKU_01,2013-04-01,89.424
12472,Agency_22,SKU_01,2013-05-01,79.164


In [4]:
data.columns

Index(['agency', 'sku', 'volume', 'date', 'industry_volume', 'soda_volume',
       'avg_max_temp', 'price_regular', 'price_actual', 'discount',
       'avg_population_2017', 'avg_yearly_household_income_2017', 'easter_day',
       'good_friday', 'new_year', 'christmas', 'labor_day', 'independence_day',
       'revolution_day_memorial', 'regional_games', 'fifa_u_17_world_cup',
       'football_gold_cup', 'beer_capital', 'music_fest',
       'discount_in_percent', 'timeseries'],
      dtype='object')

In [5]:
data['agency'].nunique()

58

In [6]:
data['sku'].nunique()

25

In [8]:
data.loc[data['time_idx'] <= 53].shape

(18900, 27)

In [9]:
class Dataset(torch.utils.data.DataLoader):
    
    def __init__(self, features, targets):
        
        self.targets = targets
        self.features = features
        
    def __len__(self):
        
        return len(self.features['time_idx'].unique())
    
    def __getitem__(self, index):
        
        print(self.features)
        
        X = self.features.loc[self.features['time_idx'] == index]
        
        Y = self.targets.loc[self.targets['time_idx'] == index]
        
        return X, Y

In [10]:
training = Dataset(data.loc[data['time_idx'] <= 52].drop(columns = 'volume'),data.loc[data['time_idx'] <= 53][['time_idx', 'volume']])

validation = Dataset(data.drop(columns = 'volume'), data[['time_idx', 'volume']])

In [11]:
train_generator = torch.utils.data.DataLoader(training)
valid_generator = torch.utils.data.DataLoader(validation)

In [13]:
training.__len__(), validation.__len__()

(53, 60)

In [15]:
training.__getitem__(1)

          agency     sku       date  industry_volume  soda_volume  \
0      Agency_22  SKU_01 2013-01-01        492612703    718394219   
238    Agency_37  SKU_04 2013-01-01        492612703    718394219   
237    Agency_59  SKU_03 2013-01-01        492612703    718394219   
236    Agency_11  SKU_01 2013-01-01        492612703    718394219   
235    Agency_05  SKU_05 2013-01-01        492612703    718394219   
...          ...     ...        ...              ...          ...   
13885  Agency_47  SKU_01 2017-05-01        628759461    917412482   
13886  Agency_60  SKU_01 2017-05-01        628759461    917412482   
13887  Agency_16  SKU_01 2017-05-01        628759461    917412482   
13888  Agency_48  SKU_28 2017-05-01        628759461    917412482   
13880  Agency_29  SKU_06 2017-05-01        628759461    917412482   

       avg_max_temp  price_regular  price_actual    discount  \
0         25.845238    1168.903668   1069.166193   99.737475   
238       26.505000    1852.273642   1611.4

(         agency     sku       date  industry_volume  soda_volume  \
 7233  Agency_01  SKU_04 2013-02-01        431937346    753938444   
 7234  Agency_09  SKU_01 2013-02-01        431937346    753938444   
 7238  Agency_44  SKU_02 2013-02-01        431937346    753938444   
 7236  Agency_44  SKU_01 2013-02-01        431937346    753938444   
 7237  Agency_33  SKU_04 2013-02-01        431937346    753938444   
 ...         ...     ...        ...              ...          ...   
 7113  Agency_22  SKU_04 2013-02-01        431937346    753938444   
 7112  Agency_03  SKU_32 2013-02-01        431937346    753938444   
 7120  Agency_49  SKU_02 2013-02-01        431937346    753938444   
 7001  Agency_12  SKU_04 2013-02-01        431937346    753938444   
 7000  Agency_15  SKU_02 2013-02-01        431937346    753938444   
 
       avg_max_temp  price_regular  price_actual    discount  \
 7233     19.984000    1728.545508   1500.037283  228.508225   
 7234     31.958696    1183.393509   1053.

In [16]:
validation.__getitem__(1)

         agency     sku       date  industry_volume  soda_volume  \
0     Agency_22  SKU_01 2013-01-01        492612703    718394219   
238   Agency_37  SKU_04 2013-01-01        492612703    718394219   
237   Agency_59  SKU_03 2013-01-01        492612703    718394219   
236   Agency_11  SKU_01 2013-01-01        492612703    718394219   
235   Agency_05  SKU_05 2013-01-01        492612703    718394219   
...         ...     ...        ...              ...          ...   
6765  Agency_08  SKU_03 2017-12-01        618073219    919709619   
6764  Agency_60  SKU_05 2017-12-01        618073219    919709619   
6763  Agency_19  SKU_05 2017-12-01        618073219    919709619   
6771  Agency_60  SKU_03 2017-12-01        618073219    919709619   
6650  Agency_56  SKU_01 2017-12-01        618073219    919709619   

      avg_max_temp  price_regular  price_actual    discount  \
0        25.845238    1168.903668   1069.166193   99.737475   
238      26.505000    1852.273642   1611.466298  240.8073

(         agency     sku       date  industry_volume  soda_volume  \
 7233  Agency_01  SKU_04 2013-02-01        431937346    753938444   
 7234  Agency_09  SKU_01 2013-02-01        431937346    753938444   
 7238  Agency_44  SKU_02 2013-02-01        431937346    753938444   
 7236  Agency_44  SKU_01 2013-02-01        431937346    753938444   
 7237  Agency_33  SKU_04 2013-02-01        431937346    753938444   
 ...         ...     ...        ...              ...          ...   
 7113  Agency_22  SKU_04 2013-02-01        431937346    753938444   
 7112  Agency_03  SKU_32 2013-02-01        431937346    753938444   
 7120  Agency_49  SKU_02 2013-02-01        431937346    753938444   
 7001  Agency_12  SKU_04 2013-02-01        431937346    753938444   
 7000  Agency_15  SKU_02 2013-02-01        431937346    753938444   
 
       avg_max_temp  price_regular  price_actual    discount  \
 7233     19.984000    1728.545508   1500.037283  228.508225   
 7234     31.958696    1183.393509   1053.